In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",      # transport: stdio, http, websocket based on MCP Server
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [8]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

In [10]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [11]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='c2bc8ef2-5cd0-480b-92e9-ecf8e2573501'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 271, 'total_tokens': 427, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D41kwzhGU6eMwhZ5NNK6bPkQpjNjM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c136c-f51e-7a90-8f4a-927854a9b8ff-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters'}, 'id': 'call_fZaTFUAi9X9PYkmpmTVzJqrn', 'type': 'tool_call'}], usage_metadata={'input_to

## Online MCP

In [20]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [21]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
)

In [22]:
question = HumanMessage(content="What time is it in Vietnam?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it in Vietnam?', additional_kwargs={}, response_metadata={}, id='3f10523b-a9a5-4451-bbec-7ce0fbec9a97'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 286, 'prompt_tokens': 298, 'total_tokens': 584, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D42GdSmGPTbDJ0EnLWEadJbalNdRN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c138a-f114-7ce1-add6-b781dcd7722c-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Ho_Chi_Minh'}, 'id': 'call_fA2FNAxVzOH76ZosMbTglxtq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 298, 'outpu

In [18]:
print(response["messages"][-1].content)

Vietnam (ICT, UTC+7) is currently Saturday, January 31, 2026, 17:11 (5:11 PM).
